In [77]:
# Import necessary libraries
import os
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import ConvLSTM2D, MaxPooling3D, TimeDistributed, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, accuracy_score, f1_score, confusion_matrix
import datetime as dt
import warnings

# Suppress TensorFlow warnings
warnings.filterwarnings("ignore")

# Define constants
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
SEQUENCE_LENGTH = 20
DATASET_DIR = '/kaggle/input/human-activity-recognition-video-dataset/Human Activity Recognition - Video Dataset'
CLASSES_LIST = ['Clapping', 'Meet and Split', 'Sitting', 'Standing Still', 'Walking', 'Walking While Reading Book', 'Walking While Using Phone']

def frames_extraction(video_path, sequence_length=SEQUENCE_LENGTH, image_height=IMAGE_HEIGHT, image_width=IMAGE_WIDTH):
    """Extract frames from a video, resizing and normalizing each frame."""
    frames_list = []
    video_reader = cv2.VideoCapture(video_path)
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count / sequence_length), 1)

    for frame_counter in range(sequence_length):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        success, frame = video_reader.read()
        if not success:
            break
        resized_frame = cv2.resize(frame, (image_width, image_height))
        normalized_frame = resized_frame / 255.0
        frames_list.append(normalized_frame)

    video_reader.release()
    return np.array(frames_list)

def create_dataset():
    """Extract frames from videos in each class and create datasets."""
    features = []
    labels = []
    video_files_paths = []

    for class_index, class_name in enumerate(CLASSES_LIST):
        print(f'Extracting Data of Class: {class_name}')
        class_dir = os.path.join(DATASET_DIR, class_name)
        if not os.path.exists(class_dir):
            print(f"Directory {class_dir} does not exist.")
            continue
        files_list = os.listdir(class_dir)
        for file_name in files_list:
            video_file_path = os.path.join(class_dir, file_name)
            frames = frames_extraction(video_file_path)
            if len(frames) == SEQUENCE_LENGTH:
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    features = np.asarray(features)
    labels = np.array(labels)
    return features, labels, video_files_paths

def create_convlstm_model(sequence_length, image_height, image_width, num_classes):
    """Create a ConvLSTM model for video classification."""
    model = Sequential()
    model.add(Input(shape=(sequence_length, image_height, image_width, 3)))
    model.add(ConvLSTM2D(filters=4, kernel_size=(3, 3), activation='tanh', recurrent_dropout=0.2, return_sequences=True, data_format='channels_last'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))
    model.add(TimeDistributed(Dropout(0.2)))
    model.add(ConvLSTM2D(filters=8, kernel_size=(3, 3), activation='tanh', recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))
    model.add(TimeDistributed(Dropout(0.2)))
    model.add(ConvLSTM2D(filters=14, kernel_size=(3, 3), activation='tanh', recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))
    model.add(TimeDistributed(Dropout(0.2)))
    model.add(ConvLSTM2D(filters=16, kernel_size=(3, 3), activation='tanh', recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))
    model.add(TimeDistributed(Dropout(0.2)))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

# Create the dataset
features, labels, video_files_paths = create_dataset()

# Convert labels to one-hot encoding
one_hot_encoded_labels = to_categorical(labels)

# Set random seeds for reproducibility
seed_constant = 27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

# Split the data into training (75%) and testing (25%) sets
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size=0.25, shuffle=True, random_state=seed_constant)

# Create the ConvLSTM model
num_classes = len(CLASSES_LIST)
convlstm_model = create_convlstm_model(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, num_classes)

# Create an instance of Early Stopping Callback
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)

# Train the model
convlstm_model_training_history = convlstm_model.fit(
    x=features_train,
    y=labels_train,
    epochs=10,
    batch_size=1,
    shuffle=True,
    validation_split=0.2,
    callbacks=[early_stopping_callback]
)

# Evaluate the model
model_evaluation_history = convlstm_model.evaluate(features_test, labels_test)
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

# Define the date and time format for saving the model file
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

# Define the model file name
model_file_name = f'convlstm_model___Date_Time_{current_date_time_string}___Loss_{model_evaluation_loss}_Accuracy_{model_evaluation_accuracy}.h5'

# Save the model
convlstm_model.save(model_file_name)

# Print a success message
print("Model Created and Saved Successfully!")


Extracting Data of Class: Clapping
Extracting Data of Class: Meet and Split
Extracting Data of Class: Sitting
Extracting Data of Class: Standing Still
Extracting Data of Class: Walking
Extracting Data of Class: Walking While Reading Book
Extracting Data of Class: Walking While Using Phone


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d_4 (ConvLSTM2D)      │ (None, 20, 62, 62, 4)  │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_4 (MaxPooling3D)  │ (None, 20, 31, 31, 4)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 20, 31, 31, 4)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_5 (ConvLSTM2D)      │ (None, 20, 29, 29, 8)  │         3,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_5 (MaxPooling3D)  │ (None, 20, 15, 15, 8)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 20, 15, 15, 8)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_6 (ConvLSTM2D)      │ (None, 20, 13, 13, 14) │        11,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_6 (MaxPooling3D)  │ (None, 20, 7, 7, 14)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, 20, 7, 7, 14)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_7 (ConvLSTM2D)      │ (None, 20, 5, 5, 16)   │        17,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_7 (MaxPooling3D)  │ (None, 20, 3, 3, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_7              │ (None, 20, 3, 3, 16)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2880)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │        20,167 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,167 (207.68 KB)

 Trainable params: 53,167 (207.68 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 204s 277ms/step - accuracy: 0.1722 - loss: 1.9687 - val_accuracy: 0.2036 - val_loss: 1.9469
Epoch 2/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 203s 279ms/step - accuracy: 0.2005 - loss: 1.9194 - val_accuracy: 0.5389 - val_loss: 1.3592
Epoch 3/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 185s 277ms/step - accuracy: 0.5969 - loss: 1.2270 - val_accuracy: 0.6946 - val_loss: 0.8301
Epoch 4/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 185s 278ms/step - accuracy: 0.8365 - loss: 0.5274 - val_accuracy: 0.8263 - val_loss: 0.5974
Epoch 5/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 190s 285ms/step - accuracy: 0.9327 - loss: 0.2331 - val_accuracy: 0.8623 - val_loss: 0.5301
Epoch 6/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 184s 276ms/step - accuracy: 0.9207 - loss: 0.2123 - val_accuracy: 0.8802 - val_loss: 0.3479
Epoch 7/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 185s 277ms/step - accuracy: 0.9496 - loss: 0.1225 - val_accuracy: 0.8922 - val_loss: 0.4320
9/9 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.8515 - loss: 0.4372
Model

In [ ]:
# Save the model in TensorFlow SavedModel format
convlstm_model.save('path/to/saved_model')


In [ ]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(convlstm_model)
tflite_model = converter.convert()
with open('path/to/model.tflite', 'wb') as f:
    f.write(tflite_model)


In [ ]:
import tf2onnx
import tensorflow as tf

# Convert the model to ONNX format
onnx_model = tf2onnx.convert.from_keras(convlstm_model)
with open('path/to/model.onnx', 'wb') as f:
    f.write(onnx_model.SerializeToString())


In [ ]:
from flask import Flask, request, jsonify
import tensorflow as tf
import numpy as np

app = Flask(__name__)
model = tf.keras.models.load_model('path/to/saved_model')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    frames = np.array(data['frames'])
    predictions = model.predict(frames)
    predicted_class = np.argmax(predictions, axis=1)
    return jsonify({'predicted_class': int(predicted_class)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


In [ ]:
# Start TensorFlow Serving with Docker
docker run -p 8501:8501 --name=tf_model_serving --mount type=bind,source=/path/to/saved_model,target=/models/model -e MODEL_NAME=model -t tensorflow/serving:latest
